In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_absolute_error

In [ ]:
df = pd.read_csv('/kaggle/input/datasciencetimeseriesforecasting/ETTh1_without_missing.csv')

In [ ]:
df['date'] = pd.to_datetime(df['date'])

df.set_index('date', inplace=True)
df = df.drop(["Id", "Unnamed: 3"], axis=1)

In [ ]:
df

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df["OT"])

In [ ]:
#df = df[df.index >= '2017-01-01']
plt.figure(figsize=(12, 6))
plt.plot(df["OT"])

In [ ]:
df

In [ ]:
def create_sequences(df, sequence_length):
    X, y = [], []
    for i in range(len(df) - sequence_length):
        X.append(df[i:(i + sequence_length)])
        y.append(df[i + sequence_length])
    return np.array(X), np.array(y)

sequence_length = 100  # Length of input sequences
train_data = df['OT'].values  # Assuming 'OT' is the target variable
X_train, y_train = create_sequences(train_data, sequence_length)


In [ ]:
model = Sequential([
    LSTM(units=64, input_shape=(X_train.shape[1], 1)),
    Dense(units=1)
])
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=15, batch_size=32)


In [ ]:
last_sequence = train_data[-sequence_length:]  # Last sequence in the dataset
predictions = []
for _ in range(100):
    input_sequence = np.array(last_sequence[-sequence_length:]).reshape(1, sequence_length, 1)
    predicted_value = model.predict(input_sequence)[0][0]
    predictions.append(predicted_value)
    last_sequence = np.append(last_sequence, predicted_value)

In [ ]:
print("Predictions for the next 100 values:", predictions)

In [ ]:
predicted_df = pd.DataFrame({'Id': range(100), 'OT': predictions})

In [ ]:
predicted_df

In [ ]:
predicted_df.to_csv('predicted_values.csv', index=False)